In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(stringr))
library(ggrepel)
library(reshape2)
library(ggsci)
library(rstatix)
library(ggpubr)
library(data.table)
library(ggtern)
library(viridis)
library(scales)
library(RColorBrewer)
library(parallel)
library(purrr)
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter



Attaching package: ‘data.table’


The following objects are masked from ‘package:reshape2’:

    dcast, melt


The following objects are masked from ‘package:dplyr’:

    between, first, last


Registered S3 methods overwritten by 'ggtern':
  method           from   
  grid.draw.ggplot ggplot2
  plot.ggplot      ggplot2
  print.ggplot     ggplot2

--
Remember to cite, run citation(package = 'ggtern') for further info.
--


Attaching package: ‘ggtern’


The following object is masked from ‘package:rstatix’:

    mahalanobis_distance


The following objects are masked from ‘package:ggplot2’:

    aes, annotate, ggplot, ggplot_build, ggplot_gtable, ggplotGrob,
    ggsave, layer_data, theme_bw, theme_classic, theme_dark,
    theme_gray, theme_light, theme_linedraw, theme_minimal, theme_void




# Read Meta Data of Selected Samples

In [2]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/SF4_meta_data-2024-05-05.csv")
meta_data$ageAtEnrollment[meta_data$ageAtEnrollment == '89+'] <- '90'
meta_data$ageAtEnrollment<-as.numeric(meta_data$ageAtEnrollment)
meta_data<-meta_data[!is.na(meta_data$ageAtEnrollment),]
meta_data$Ages<-meta_data$ageAtEnrollment
meta_data <- meta_data[!duplicated(meta_data$sample.sampleKitGuid, fromLast = TRUE), ]
meta_data$CMV<-meta_data$lab.revisionHistory.dataHistory.CMV.Ab.Screen.Result..CMVRES..newValue

# Composite Score

In [3]:
file_list<-paste0("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/SF4/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/", meta_data$pbmc_sample_id, ".csv")

In [4]:
df_list<-read_pseudobulk_expression(file_list, mc_cores = 60)

[1] "Total reading time: 11.849 seconds"
[1] "The length of the list matches the length of the input path."


### DESEQ2 result

In [7]:
deseq2_result_Y1D0<-read.csv('/home/jupyter/BRI_Figures_Final_V2/Figure2/02_DEG/Deseq2_Result_Y1D0.csv')

In [8]:
# filter significant genes 
deseq2_result_Y1D0_AgeGroup_sig<-deseq2_result_Y1D0 %>% 
  filter(contrast == "cohort.cohortGuid") %>%
  filter(padj < 0.05,
         abs(log2FoldChange) > 0.1)

### Get Composite Score

In [9]:
df_degs_counts<-as.data.frame(table(deseq2_result_Y1D0_AgeGroup_sig$celltype,deseq2_result_Y1D0_AgeGroup_sig$Direction))
df_degs_counts_filtered<-df_degs_counts %>% arrange(Var1)   %>% filter(Freq>20)

In [10]:
score_df_list <- mclapply(unique(df_degs_counts_filtered$Var1), function(celltype_single) {
    top_n <- 100
    Direction_of_DEG <- "HigherInBR2"
    
    selected_genes <- deseq2_result_Y1D0_AgeGroup_sig %>%
        filter(celltype == celltype_single, Direction == Direction_of_DEG) %>%
        arrange(padj, desc(abs(log2FoldChange))) %>%
        slice_head(n = top_n) %>%
        select(gene) %>%
        pull()
    
    long_format <- filter_genes_and_celltype(df_list, selected_genes, celltype_single, longformat = TRUE) %>%
        left_join(meta_data) %>%
        filter(!is.na(Mean_Expression)) %>%
        group_by(Gene) %>%
        mutate(Z_score_Mean_Expression = scale(Mean_Expression)) %>%
        ungroup()
    
    composite_score_df <- long_format %>%
        group_by(pbmc_sample_id) %>%
        summarise(
            composite_score_sum_mean = sum(Mean_Expression, na.rm = TRUE),
            composite_score_sum_scaled_mean = sum(Z_score_Mean_Expression, na.rm = TRUE)
        ) %>%
        left_join(meta_data) %>%
        mutate(celltype = celltype_single) %>%
        as.data.frame()
    
    return(composite_score_df)
}, mc.cores = 10)


In [11]:
composite_score_df_all<-do.call(rbind,score_df_list)

In [12]:
composite_score_df_all<-composite_score_df_all %>% filter (celltype=="CM CD4 T cell")

In [13]:
composite_score_variable<-composite_score_df_all[c("subject.subjectGuid","composite_score_sum_scaled_mean")]

# NMF scores

In [14]:
df_list<-mclapply(meta_data$pbmc_sample_id,function(x){
df<-read.csv(paste0("/home/jupyter/BRI_Figures_Final_V2/Figure6/02_NMF_CD4_T_cell_Projection/NMF_Score_SF4/",x,".csv"))
df$pbmc_sample_id=x
return(df)
},mc.cores=60)

In [15]:
df_combined<-do.call(rbind,df_list)

In [16]:
df_combined_longformat<- pivot_longer(df_combined, cols = starts_with("NMF"), names_to = "NMF_Scores", values_to = "value")

In [17]:
df_combined_longformat<-df_combined_longformat %>% group_by(pbmc_sample_id,NMF_Scores,AIFI_L3) %>% summarise(AverageScore=mean(value))

`summarise()` has grouped output by 'pbmc_sample_id', 'NMF_Scores'. You can
override using the `.groups` argument.


In [18]:
df_combined_longformat<-left_join(df_combined_longformat,meta_data) %>% ungroup()

Joining with `by = join_by(pbmc_sample_id)`


In [19]:
df_combined_longformat_filtered_TH2<-df_combined_longformat %>% filter(NMF_Scores %in% c('NMF5_Th2'))%>% 
                           filter(AIFI_L3 %in% c('CM CD4 T cell')) %>% select(subject.subjectGuid,AverageScore) %>% as.data.frame()
df_combined_longformat_filtered_Tfh<-df_combined_longformat %>% filter(NMF_Scores %in% c('NMF6_Tfh'))%>% 
                           filter(AIFI_L3 %in% c('CM CD4 T cell'))%>% select(subject.subjectGuid,AverageScore)%>% as.data.frame()

In [20]:
colnames(df_combined_longformat_filtered_TH2)<-c("subject.subjectGuid","NMF_Th2")
colnames(df_combined_longformat_filtered_Tfh)<-c("subject.subjectGuid","NMF_Tfh")

In [21]:
nmf_variable<-left_join(df_combined_longformat_filtered_TH2,df_combined_longformat_filtered_Tfh)

Joining with `by = join_by(subject.subjectGuid)`


# cell to cell interaction scores

In [23]:
combined_significant_means<-read.csv("cellphonedb_SF4/combined_statistical_analysis_significant_means.csv",check.names = FALSE)

In [24]:
combined_significant_means_long<-pivot_longer(
  combined_significant_means,
  cols = matches("\\|"), # Select columns with pattern "|"
  names_to = c("variable"),
  values_to = "mean_interaction"
)

In [25]:
combined_significant_means_long<-combined_significant_means_long[!is.na(combined_significant_means_long$mean_interaction),]

In [26]:
combined_significant_means_long<-combined_significant_means_long %>% filter(pbmc_sample_id%in% meta_data$pbmc_sample_id)%>% filter(pbmc_sample_id%in% meta_data$pbmc_sample_id)

In [27]:
combined_significant_means_long<-left_join(combined_significant_means_long,meta_data)

Joining with `by = join_by(pbmc_sample_id)`


In [28]:
CD40LG_CD40_MEAN<-combined_significant_means_long %>% filter(interacting_pair=="CD40LG_CD40",
                                                      variable=="CM CD4 T cell|Core memory B cell")%>% 
select(subject.subjectGuid,mean_interaction)%>% as.data.frame()
PTPRC_CD22_MEAN<-combined_significant_means_long %>% filter(interacting_pair=="PTPRC_CD22",
                                                      variable=="CM CD4 T cell|Core memory B cell")%>% 
select(subject.subjectGuid,mean_interaction)%>% as.data.frame()

In [29]:
colnames(CD40LG_CD40_MEAN)<-c("subject.subjectGuid","InteractionScore_CD40LG_CD40")
colnames(PTPRC_CD22_MEAN)<-c("subject.subjectGuid","InteractionScore_PTPRC_CD22")

In [30]:
c2c_interaction_variable<-left_join(CD40LG_CD40_MEAN,PTPRC_CD22_MEAN)

Joining with `by = join_by(subject.subjectGuid)`


# Mean Expression

In [31]:
file_list<-paste0("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/SF4/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/", meta_data$pbmc_sample_id, ".csv")
df_list<-read_pseudobulk_expression(file_list)

[1] "Total reading time: 13.117 seconds"
[1] "The length of the list matches the length of the input path."


In [32]:
selected_genes<-c("CD40LG","PTPRC","CXCR5")


In [33]:
df<-filter_genes_and_celltype(df_list, selected_genes, c('CM CD4 T cell'), longformat = TRUE)


Warning message:
“The melt generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is superseded and is no longer actively developed, and this redirection is now deprecated. To continue using melt methods from reshape2 while both libraries are attached, e.g. melt.list, you can prepend the namespace, i.e. reshape2::melt(exp_matrix). In the next version, this warning will become an error.”


In [34]:
df<-left_join(df,meta_data)

Joining with `by = join_by(pbmc_sample_id)`


In [35]:
expression_CD40LG<-df %>% filter(Gene=="CD40LG") %>% 
select(subject.subjectGuid,Mean_Expression)%>% as.data.frame()
expression_PTPRC<-df %>% filter(Gene=="PTPRC") %>% 
select(subject.subjectGuid,Mean_Expression)%>% as.data.frame()
expression_CXCR5<-df %>% filter(Gene=="CXCR5") %>% 
select(subject.subjectGuid,Mean_Expression)%>% as.data.frame()

In [36]:
colnames(expression_CD40LG)<-c("subject.subjectGuid","Mean_Expression_CD40LG")
colnames(expression_PTPRC)<-c("subject.subjectGuid","Mean_Expression_PTPRC")
colnames(expression_CXCR5)<-c("subject.subjectGuid","Mean_Expression_CXCR5")

In [37]:
mean_expression_variable<-reduce(list(expression_CD40LG,expression_PTPRC,expression_CXCR5), full_join, by = "subject.subjectGuid")

# Combine all together

In [39]:
combined_df <- reduce(list(nmf_variable,
                           composite_score_variable,
                           c2c_interaction_variable,
                           mean_expression_variable
                           ), left_join, by = "subject.subjectGuid")

In [40]:
write.csv(combined_df,"SF4_Tcell_data_forcorrelation.csv")